# Retrieve CMV and BMI from HISE

The Python version of the HISE SDK allows retrieval of sample metadata regardless of pipeline sample processing. We'll use this SDK to pull these results.

## Load packages

datetime: Used to add today's date to our output files  
hisepy: the HISE SDK  
os: Operating System files (used to make an output folder)  
pandas: DataFrames for Python  
session_info: displays the versioning Python and all of the packages we used  
warnings: Used to suppress some annoying warnings that don't impact data retrieval

In [1]:
from datetime import date
import hisepy
import os
import pandas
import session_info

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Read sample metadata from HISE

In [2]:
sample_meta_file_uuid = '223b4aa9-19fc-41e1-8bea-43682e5ac278'
sample_meta_file_name = 'ref_h5_meta_data_2024-02-08.csv'
file_query = hisepy.reader.download_files(
    {sample_meta_file_uuid: sample_meta_file_name}
)

In [3]:
sample_meta_file = 'cache/downloadable/' + sample_meta_file_name
sample_meta = pandas.read_csv(sample_meta_file)

## Query HISE to get data

First, we make a dictionary (with curly braces) that defines what we want to get. In this case, we want to get all of the samples from the BR2 cohort. In this query, cohort is found as 'cohortGuid'.

Each entry in the dictionary has to be a list (square braces), even if it has a single entry.

In [4]:
subject_ids = sample_meta['subject.subjectGuid'].tolist()
query_dict = {
    'subjectGuid': subject_ids
}

Now, we send this dictionary to HISE via hisepy

In [5]:
sample_data = hisepy.reader.read_samples(
    query_dict = query_dict
)

What we get back is a dictionary containing multiple kinds of information.

We can see what these are called with they `.keys()` method:

In [6]:
sample_data.keys()

dict_keys(['metadata', 'specimens', 'survey', 'labResults'])

CMV and the weight and height data we'll need for our analysis are available in labResults.

CMV status is stored as 'CMV IgG Serology Result Interpretation' or 'CMV Ab Screen Result'.  

Height and weight are simply 'Height' and 'Weight'.

Let's select columns from these results to get just subject, sample kit, CMV status, height, and weight.

For CMV status, the results can be assigned to different samples, and to different columns. We'll select only rows where there's a result and then check that we have results for all of our subjects.

In [7]:
keep_columns = ['subjectGuid', 'CMV IgG Serology Result Interpretation', 'CMV Ab Screen Result']
cmv_data = sample_data['labResults'][keep_columns]

Keep rows where either of the result columns have Positive or Negative values:

In [12]:
keep_rows = []
keep_values = ['Negative','Positive']
for i in range(cmv_data.shape[0]):
    ig = cmv_data['CMV IgG Serology Result Interpretation'][i]
    ab = cmv_data['CMV Ab Screen Result'][i]
    if (ig in keep_values) or (ab in keep_values):
        keep_rows.append(True)
    else:
        keep_rows.append(False)

In [13]:
cmv_data = cmv_data.iloc[keep_rows,:]

Keep rows where the subjects correspond to the subjects under study:

In [20]:
keep_rows = cmv_data['subjectGuid'].isin(sample_meta['subject.subjectGuid'])
cmv_data = cmv_data.iloc[keep_rows.tolist(),:]

In [21]:
cmv_data.shape

(115, 3)

Some subjects have multiple entries. We'll combine these, and assign Positive if any entry is Positive, and Negative otherwise.

In [31]:
ig_results = []
ab_results = []
cmv_results = []
for s in sample_meta['subject.subjectGuid'].tolist():
    sample_cmv = cmv_data.iloc[cmv_data.subjectGuid.isin([s]).tolist(),:]
    ig = sample_cmv['CMV IgG Serology Result Interpretation'].tolist()
    ab = sample_cmv['CMV Ab Screen Result'].tolist()
    if 'Positive' in ig:
        ig_results.append('Positive')
    elif 'Negative' in ig:
        ig_results.append('Negative')
    else:
        ig_results.append('NA')

    if 'Positive' in ab:
        ab_results.append('Positive')
    elif 'Negative' in ab:
        ab_results.append('Negative')
    else:
        ab_results.append('NA')

    both = ig + ab
    if 'Positive' in both:
        cmv_results.append('Positive')
    else:
        cmv_results.append('Negative')

In [32]:
cmv_df = pandas.DataFrame(
    {
        'subject.subjectGuid': sample_meta['subject.subjectGuid'].tolist(),
        'CMV IgG Serology Result Interpretation': ig_results,
        'CMV Ab Screen Result': ab_results,
        'subject.cmv': cmv_results
    }
)

In [34]:
cmv_file = 'output/ref_subject_cmv_lab_results_{date}.csv'.format(date = date.today())
cmv_df.to_csv(cmv_file)

Height and weight can be selected using the specific sampleKitGuid that corresponds to the samples in our reference.

In [47]:
keep_columns = ['subjectGuid', 'sampleKitGuid', 'Height', 'Weight']
hw_data = sample_data['labResults'][keep_columns]

In [48]:
keep_rows = hw_data['sampleKitGuid'].isin(sample_meta['sample.sampleKitGuid']).tolist()
hw_data = hw_data.iloc[keep_rows,:]

There are some duplicate entries with missing values. We'll remove these.

In [51]:
keep_rows = [not x for x in hw_data['Height'].isnull()]
hw_data = hw_data.iloc[keep_rows,:]

In [54]:
hw_data.shape

(104, 4)

Based on the number of remaining rows, we'll be missing values for 4 samples.

We can compute BMI based on Height (in cm) and weight (in kg):  
BMI = weight / (height^2) where weight is in kg and height is in meters.  
So, we'll use BMI = Weight / ( (Height / 100)^2 ) to account for the use of cm in height.

In [82]:
h_results = []
w_results = []
bmi_results = []
for s in sample_meta['subject.subjectGuid'].tolist():
    sample_hw = hw_data.iloc[hw_data.subjectGuid.isin([s]).tolist(),:]

    if sample_hw.shape[0] == 0:
        h_results.append('NA')
        w_results.append('NA')
        bmi_results.append('NA')
    else:
        h = sample_hw['Height'].tolist()[0]
        if h == '':
            h_results.append('NA')
        else:
            h = float(h)
            h_results.append(h)
        
        w = sample_hw['Weight'].tolist()[0]
        if w == '':
            w_results.append('NA')
        else:
            w = float(w)
            w_results.append(w)

        if isinstance(h, str) | isinstance(w, str):
            bmi_results.append('NA')
        else:
            bmi = w / ( pow(h / 100,2) )
            bmi_results.append(bmi)

In [83]:
len(bmi_results)

108

In [88]:
hw_df = pandas.DataFrame(
    {
        'subject.subjectGuid': sample_meta['subject.subjectGuid'].tolist(),
        'sample.sampleKitGuid': sample_meta['sample.sampleKitGuid'].tolist(),
        'Height': h_results,
        'Weight': w_results,
        'subject.bmi': bmi_results
    }
)

In [95]:
hw_df = hw_df.sort_values('subject.subjectGuid')

In [14]:
out_dir = 'output'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [96]:
hw_file = 'output/ref_subject_bmi_results_{date}.csv'.format(date = date.today())
hw_df.to_csv(hw_file)

## Downstream use

For later use, we'll load these files into HISE using a Watchfolder.

This is because HISE doesn't have a great way (yet) to track the provenance of clinical lab values. For now, we can supply these files to HISE through our Project Store for downstream steps.

In [18]:
session_info.show()